In [ ]:
# ACCESS TOKENとREFRESH TOKENが変更されてしまうので、自動化する

import gather_keys_oauth2 as gk # fitbit提供

# ここを変更する
CLIENT_ID =  "XXXXXX"
CLIENT_SECRET  = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

# Authorization
server=gk.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()

# profile取得
profile = server.fitbit.user_profile_get()
# print(profile) #やばい

# 名前を表示
print('You are authorized to access data for the user: {}'.format(
    profile['user']['fullName']))

# ACCESS TOKENとREFRESH TOKENはここにある
print('TOKEN\n=====\n')
for key, value in server.fitbit.client.session.token.items():
    print('{} = {}'.format(key, value))

In [ ]:
import fitbit
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime
%matplotlib inline

# メモしたID等
ACCESS_TOKEN =  server.fitbit.client.session.token["access_token"]
REFRESH_TOKEN =  server.fitbit.client.session.token["refresh_token"]

# 取得したい日付
DATE = "2018-03-18"
# DATE = str(datetime.date.today())

# ID等の設定
authd_client = fitbit.Fitbit(CLIENT_ID, CLIENT_SECRET, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN)

# 心拍数を取得（1秒単位）
data_sec = authd_client.intraday_time_series('activities/heart', DATE, detail_level='1sec') #'1sec', '1min', or '15min'
heart_sec = data_sec["activities-heart-intraday"]["dataset"]
heart_sec[:10]

# Pandasを使用
heart_df = pd.DataFrame.from_dict(heart_sec)
heart_df.index = pd.to_datetime([DATE + " " + t for t in heart_df.time])

#データ表示
heart_df.plot(y="value", figsize=(20,5))


# 5行分表示
heart_df.head()

In [ ]:
fig = plt.figure(figsize=(15,5))
plt.plot(heart_df.value)
plt.title("Heart rate (time series)")
plt.xlabel("Time")
plt.ylabel("Heart rate[bpm]")
plt.savefig('hr.png')

In [ ]:
plt.hist(heart_df.value)
plt.title("Histogram of heart rate")
plt.xlabel("Heart rate[bpm]")
plt.ylabel("Frequency[times]")
plt.savefig('hist.png')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

# 箱ひげ図をつくる
bp = ax.boxplot(heart_df.value)
sigma = np.std(heart_df.value)# 標準偏差を求める
mean = np.mean(heart_df.value)# 平均を求める
plt.title("sigma="+str(sigma)+", average="+str(mean))
plt.ylabel("Heart rate [bpm]")
plt.savefig('hige.png')


In [ ]:
plt.scatter(heart_df.value[:-1],heart_df.value[1:])
plt.xlabel("HR $\it{_{n}}$ [bpm]")
plt.ylabel("HR $\it{_{n+1}}$ [bpm]")
plt.savefig('lorenz_plot.png')